In [1]:
import pandas as pd

# Load surf spot data
surf_spots = pd.read_csv("Surf Spots.csv")
print(surf_spots.head())

            Name  Latitude  Longitude       Direction         Type  \
0     Arugam Bay  6.840739  81.836999           Right  Point-break   
1      Dalawella  5.999671  80.263658  Right and left   Reef-coral   
2    Devils rock  5.962863  80.379945            Left   Reef-rocky   
3  Dewata, Galle  6.032651  80.238465  Right and left  Point-break   
4  Elephant rock  6.810079  81.824244           Right  Point-break   

            Experience  
0          All surfers  
1  Experienced surfers  
2  Experienced surfers  
3          All surfers  
4          All surfers  


In [3]:
import ee
import geemap

# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize()

In [5]:
# Convert surf spots to GEE FeatureCollection
features = []
for _, row in surf_spots.iterrows():
    point = ee.Geometry.Point([row['Longitude'], row['Latitude']])
    feature = ee.Feature(point, {'surf_spot_name': row['Name']})
    features.append(feature)

surf_spots_fc = ee.FeatureCollection(features)

In [7]:
# Load Sentinel-1 data
sentinel1 = ee.ImageCollection("COPERNICUS/S1_GRD") \
    .filterBounds(surf_spots_fc) \
    .filterDate("2023-01-01", "2023-12-31") \
    .filter(ee.Filter.eq("instrumentMode", "IW")) \
    .select(["VV", "VH"])

In [9]:
# Function to extract Sentinel-1 data for each surf spot and date
def extract_sentinel_data_per_date(image):
    # Extract the date from the image
    image_date = image.date().format("YYYY-MM-dd")
    
    def map_feature(feature):
        geom = feature.geometry()
        # Extract mean VV and VH values for the given surf spot
        values = image.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=geom,
            scale=10,  # Sentinel-1 resolution
            maxPixels=1e9
        )
        # Add the date and VV/VH values to the feature
        return feature.set(values).set({"date": image_date})
    
    # Map the data extraction to each surf spot in the collection
    return surf_spots_fc.map(map_feature)

In [11]:
# Map the function over the Sentinel-1 ImageCollection to extract data for all dates
surf_spots_with_data = sentinel1.map(extract_sentinel_data_per_date).flatten()

In [13]:
# Clean the feature properties to keep only relevant columns
def clean_feature(feature):
    return feature.select(["surf_spot_name", "date", "VV", "VH"])

surf_spots_cleaned = surf_spots_with_data.map(clean_feature)

In [15]:
# Export the data to Google Drive
task = ee.batch.Export.table.toDrive(
    collection=surf_spots_cleaned,
    description="SurfSpots_Sentinel1_WithDates",
    fileFormat="CSV"
)
task.start()

print("Export task started. Check your Google Drive for the output file.")

Export task started. Check your Google Drive for the output file.
